In [2]:
import pandas as pd

### Quick look at the data

In [4]:
non_compliance_data = pd.read_csv('./data/afsa/non-compliance-in-personal-insolvencies.csv')
non_compliance_data.head(3)

/home/nic/code/govhack2018-EZ0/venv/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (17,18,19,21,22) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Unique ID,Calendar Year of Insolvency,SA3 of Debtor,SA3 Code of Debtor,GCCSA of Debtor,GCCSA Code of Debtor,State of Debtor,Sex of Debtor,Family Situation,Debtor Occupation Code (ANZSCO),...,Debtor Income,Primary Income Source,Unsecured Debts,Value of Assets,Type of Party,Non-Compliance Type,Result of Non-Compliance,Number of Instances,Outcome of Non-Compliance,Non-Compliance Conviction Result
0,3452750,2010,South Canberra,80106,Australian Capital Territory,8ACTE,Australian Capital Territory,Female,Single with Dependants,39.0,...,$0-$49999,Government benefits/Pensions,$0-$49999,$0-$49999,NaN,NaN,NaN,NaN,NaN,NaN
1,3563908,2011,Weston Creek,80108,Australian Capital Territory,8ACTE,Australian Capital Territory,Male,Couple without Dependants,84.0,...,$0-$49999,Government benefits/Pensions,$0-$49999,$0-$49999,NaN,NaN,NaN,NaN,NaN,NaN
2,3252673,2012,Tuggeranong,80107,Australian Capital Territory,8ACTE,Australian Capital Territory,Female,Single with Dependants,11.0,...,$50000-$99999,Gross Wages and Salary,$50000-$99999,$0-$49999,NaN,NaN,NaN,NaN,NaN,NaN


### 1. Find all unique family situations in non-compliance-in-personal-insolvencies.csv

In [12]:
non_compliance_data['Family Situation'].unique()

array(['Single with Dependants', 'Couple without Dependants',
       'Couple with Dependants', 'Single without Dependants', 'Unknown',
       'Not Stated'], dtype=object)

### 2. Express each such family situation in terms of the columns in B25 / G25

In [6]:
g25 = pd.read_csv('./data/census/gcpsa3/2016 Census GCP Statistical Area 3 for AUST/2016Census_G25_AUS_SA3.csv')
g25.head(20)

,SA3_CODE_2016,CF_no_children_F,CF_no_children_P,CF_ChU15_a_DSs_a_NdCh_F,CF_ChU15_a_DSs_a_NdCh_P,CF_ChU15_a_DSs_a_no_NdCh_F,CF_ChU15_a_DSs_a_no_NdCh_P,CF_ChU15_a_no_DSs_a_NdCh_F,CF_ChU15_a_no_DSs_a_NdCh_P,CF_ChU15_a_no_DSs_a_no_NdCh_F,...,OPF_no_ChU15_no_DSs_a_NdCh_F,OPF_no_ChU15_no_DSs_a_NdCh_P,OPF_no_ChU15_a_Total_F,OPF_no_ChU15_a_Total_P,OPF_Total_F,OPF_Total_P,Other_family_F,Other_family_P,Total_F,Total_P
0,10102,5577,10999,105,573,762,3495,185,844,3963,...,722,1562,994,2231,2072,5214,180,370,15092,44276
1,10103,2220,4334,29,151,180,803,34,157,1057,...,259,547,355,752,682,1712,52,103,4808,13190
2,10104,10377,20461,95,535,566,2675,204,922,2988,...,1026,2105,1387,2936,2929,7522,195,399,19216,50111
3,10105,3922,7788,70,397,345,1603,137,650,1861,...,554,1167,733,1565,1551,4039,129,263,9184,25950
4,10106,4093,8081,83,441,450,2102,126,575,2160,...,448,927,608,1317,1350,3588,118,240,9571,27619
5,10201,17609,34950,299,1615,1841,8465,557,2518,9973,...,3141,6701,4298,9582,7926,20155,594,1227,45495,130513
6,10202,15886,31785,360,2022,1582,7442,698,3209,8658,...,3236,7014,4236,9551,8939,23703,607,1253,42378,122809
7,10301,4509,8918,91,501,573,2665,145,684,2616,...,601,1272,857,1892,2019,5336,138,286,11480,33318
8,10302,6247,12359,100,564,518,2430,191,878,2869,...,823,1742,1065,2328,2391,6384,224,461,14100,39889
9,10303,4947,9802,63,357,433,1975,149,677,2533,...,704,1475,912,1980,1996,5205,154,322,11638,32821


In [13]:
persons_columns = list(filter(lambda e: e.endswith('_P'), g25.columns))
display(persons_columns)
display(len(persons_columns))

['CF_no_children_P',
 'CF_ChU15_a_DSs_a_NdCh_P',
 'CF_ChU15_a_DSs_a_no_NdCh_P',
 'CF_ChU15_a_no_DSs_a_NdCh_P',
 'CF_ChU15_a_no_DSs_a_no_NdCh_P',
 'CF_ChU15_a_Total_P',
 'CF_no_ChU15_a_DSs_a_NdCh_P',
 'CF_no_ChU15_a_DSs_a_no_NdCh_P',
 'CF_no_ChU15_a_no_DSs_a_NdCh_P',
 'CF_no_ChU15_a_Total_P',
 'CF_Total_P',
 'OPF_ChU15_a_DSs_a_NdCh_P',
 'OPF_ChU15_a_DSs_a_no_NdCh_P',
 'OPF_ChU15_a_no_DSs_a_NdCh_P',
 'OPF_ChU15_a_no_DSs_no_NdCh_P',
 'OPF_ChU15_a_Total_P',
 'OPF_no_ChU15_a_DSs_a_NdCh_P',
 'OPF_no_ChU15_a_DSs_no_NdCh_P',
 'OPF_no_ChU15_no_DSs_a_NdCh_P',
 'OPF_no_ChU15_a_Total_P',
 'OPF_Total_P',
 'Other_family_P',
 'Total_P']

23

In [10]:
a = ['CF_no_children_P',
 'CF_ChU15_a_DSs_a_NdCh_P',
 'CF_ChU15_a_DSs_a_no_NdCh_P',
 'CF_ChU15_a_no_DSs_a_NdCh_P',
 'CF_ChU15_a_no_DSs_a_no_NdCh_P',
 'CF_ChU15_a_Total_P',
 'CF_no_ChU15_a_DSs_a_NdCh_P',
 'CF_no_ChU15_a_DSs_a_no_NdCh_P',
 'CF_no_ChU15_a_no_DSs_a_NdCh_P',
 'CF_no_ChU15_a_Total_P',
 'CF_Total_P',
 'OPF_ChU15_a_DSs_a_NdCh_P',
 'OPF_ChU15_a_DSs_a_no_NdCh_P',
 'OPF_ChU15_a_no_DSs_a_NdCh_P',
 'OPF_ChU15_a_no_DSs_no_NdCh_P',
 'OPF_ChU15_a_Total_P',
 'OPF_no_ChU15_a_DSs_a_NdCh_P',
 'OPF_no_ChU15_a_DSs_no_NdCh_P',
 'OPF_no_ChU15_no_DSs_a_NdCh_P',
 'OPF_no_ChU15_a_Total_P',
 'OPF_Total_P',
 'Other_family_P', 'Total_P']

b = ['Couple family with no children', ''